<a href="https://colab.research.google.com/github/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [0]:
# Write your code here

# open and read file
import pandas as pd

with open("Joke_reviews2.csv", "r", encoding='utf-8') as file:
  data = pd.read_csv(file)
data["cleaned_content"]
for line in data["cleaned_content"]:
  for word in line


['i', 'person', 'saw', 'hype', 'claim', 'masterpiec', 'overreact', 'overblown', 'excit', 'anoth', 'joker', 'base', 'film', 'i', 'thought', 'look', 'solid', 'best', 'even', 'bit', 'pretenti', 'trailer', 'say', 'i', 'incred', 'wrong', 'thi', 'massiv', 'achiev', 'cinema', "'s", 'extrem', 'rare', 'day', 'age', 'cgi', 'nonsens', 'reboot', 'while', 'somewhat', 'reboot', 'sort', 'standalon', 'origin', 'tale', 'impecc', 'start', 'finish', 'echo', 'resembl', 'best', 'joker', 'origin', 'comic', 'past', 'joaquin', 'bleed', 'sweat', 'cri', 'everi', 'drop', 'magnific', 'dedic', 'perform', 'heath', 'ledger', 'would', 'proud', 'thi', 'undoubtedli', 'greatest', 'act', 'perform', 'sinc', 'heath', "'s", 'joker', 'the', 'direct', 'write', 'slickli', 'brilliant', 'bleak', 'set', 'tone', 'palpabl', 'throughout', 'when', 'film', 'place', 'blown', 'away', 'everi', 'audienc', 'member', 'awestruck', 'wit', 'film', 'could', 'still', 'transport', 'charact', "'s", 'world', 'exist', 'believ', 'hype', 'thi', 'go', 

In [0]:
 # Do Ngram (n=3)
from textblob import TextBlob
TextBlob(data["cleaned_content"]).ngram(3)


TypeError: ignored

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [0]:
# Write your code here





# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: 